## Imports

In [1]:
import torch
from torch import nn
#import torch.nn.functional as F
#import csv
#import pandas as pd

## Model Class

In [2]:
from architecture import VAE

## Training Parameters

In [3]:
# Device init
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

W_DECAY = 1e-4
LEARN_RATE = 1e-5
EPOCH_NUM = 5
BATCH_SIZE = 16

BETA = 2e-1 # for the KL divergence term

## Data Creation

In [4]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as transforms

def create_data():
    # ---INITIALIZE DATASET ---
    #Convert pilimage dataset to a standart numpy dataset
    dataset = MNIST(
        root='./data',
        download=True,  # Add this to download the dataset if needed
        transform= transforms.ToTensor()
    )

    TRIM_LEN = int(20_000)  # 60,000 - 20,000 = 40,000 SAMPLES
    TRAIN_PORTION = 0.9 # 90% training 10% everything else
    TRAIN_LEN = int((len(dataset) - TRIM_LEN) * TRAIN_PORTION)
    
    # ---SPLIT DATASET---
    train_ds, test_ds, _ = random_split(
        dataset,  # Split the dataset, not the dataloader!
        [TRAIN_LEN, len(dataset) - TRIM_LEN - TRAIN_LEN, TRIM_LEN]
    )
    #print(f"train length: {len(train_ds)} test_length: {len(test_ds)}")
    
    # ---CREATE DATALOADERS from the split datasets---
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    return train_loader, test_loader

## Training

In [5]:
def train_vae(model, train_loader, optimizer, epoch_num):
    # Method 1: One-liner
    #single_batch = next(iter(train_loader))[0].to(device)
    loss_func = nn.MSELoss()
    

    for iter, single_batch in enumerate(train_loader):
        single_batch = single_batch[0].to(device)
        # ---------Feed Forward---------
        # Extract just the generated images for now
        mean,log_var,img_gen_batch = model.forward(single_batch)

        #---------Back Prop---------
        # Loss is calculated by the batch's mean
        
        flat_sample = torch.flatten(single_batch,start_dim=1)
        print(f"flat_sample shape: {flat_sample.shape}")

        img_gen_batch = torch.flatten(img_gen_batch,start_dim=-1)
        print(f"image_gen_batch shape: {img_gen_batch.shape}")

        kl_div = -0.5 * torch.sum(
            1 + log_var - mean.pow(2) - log_var.exp(),
            dim=1
        ).mean()
        loss = loss_func(img_gen_batch, flat_sample) + BETA*kl_div

        optimizer.zero_grad()
        print(f"batch num {iter}: {loss.item()} at epoch: {epoch_num+1}")
        loss.backward()
        optimizer.step()

        
    torch.save(model.state_dict(), "vae_model.pth")
        

In [6]:
# Initialize the model
my_vae = VAE().to(device)
optim = torch.optim.Adam(params=my_vae.parameters(),
                         lr = LEARN_RATE, weight_decay=W_DECAY)

train_loader,test_loader = create_data()


for i in range(EPOCH_NUM):
    train_vae(model=my_vae, train_loader=train_loader,
            optimizer=optim,epoch_num=i)

c:\Users\orian\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
c:\Users\orian\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([16, 784])) that is different to the input size (torch.Size([1, 16, 784])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 0: 0.9030548930168152 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 1: 0.9008919596672058 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 2: 0.8998936414718628 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 3: 0.9005904793739319 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 4: 0.9002931714057922 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_gen_batch shape: torch.Size([1, 16, 784])
batch num 5: 0.9009275436401367 at epoch: 1
torch.Size([16, 784])
flat_sample shape: torch.Size([16, 784])
image_g